# ClyptQ Quickstart

Run your first backtest: load crypto data, create momentum strategy, analyze results.

In [1]:
from datetime import timedelta
from clyptq import CostModel, Constraints
from clyptq.data.loaders.ccxt import load_crypto_data
from clyptq.trading.engine import BacktestEngine
from clyptq.trading.execution import BacktestExecutor
from clyptq.trading.factors.library.momentum import MomentumFactor
from clyptq.trading.factors.library.volatility import VolatilityFactor
from clyptq.trading.portfolio.constructors import TopNConstructor
from clyptq.trading.strategy.base import SimpleStrategy

## 1. Define Strategy

Momentum strategy: buy top 3 performers, avoid high volatility

In [2]:
class MomentumStrategy(SimpleStrategy):
    def __init__(self):
        factors = [
            MomentumFactor(lookback=30),
            VolatilityFactor(lookback=30),
        ]
        constraints = Constraints(
            max_position_size=0.40,
            max_gross_exposure=1.0,
            min_position_size=0.10,
            max_num_positions=5,
        )
        super().__init__(
            factors_list=factors,
            constructor=TopNConstructor(top_n=3),
            constraints_obj=constraints,
            schedule_str="weekly",
            warmup=35,
            name="Momentum",
        )

## 2. Load Data

Download 720 days of daily OHLCV data for 5 symbols

In [3]:
symbols = ["BTC/USDT", "ETH/USDT", "BNB/USDT", "SOL/USDT", "ADA/USDT"]
store = load_crypto_data(
    symbols=symbols,
    exchange="binance",
    timeframe="1d",
    days=720
)

date_range = store.get_date_range()
print(f"Data range: {date_range.start.date()} to {date_range.end.date()}")
print(f"Total days: {(date_range.end - date_range.start).days}")

Data range: 2024-01-16 to 2026-01-04
Total days: 719


## 3. Run Backtest

Backtest with realistic costs (0.1% fees + 5bps slippage)

In [4]:
strategy = MomentumStrategy()
cost_model = CostModel(maker_fee=0.001, taker_fee=0.001, slippage_bps=5.0)
executor = BacktestExecutor(cost_model)

engine = BacktestEngine(
    strategy=strategy,
    data_store=store,
    executor=executor,
    initial_capital=10000.0,
)

start = date_range.end - timedelta(days=365)
end = date_range.end

result = engine.run(start=start, end=end, verbose=True)

Fill rejected: Insufficient cash for SOL/USDT: need 3469.85, have 3466.41


## 4. Analyze Results

Performance metrics and trade statistics

In [5]:
m = result.metrics

print("=" * 70)
print("PERFORMANCE SUMMARY")
print("=" * 70)
print(f"Total Return:       {m.total_return:>10.2%}")
print(f"Annualized Return:  {m.annualized_return:>10.2%}")
print(f"Sharpe Ratio:       {m.sharpe_ratio:>10.3f}")
print(f"Sortino Ratio:      {m.sortino_ratio:>10.3f}")
print(f"Max Drawdown:       {m.max_drawdown:>10.2%}")
print(f"Volatility:         {m.volatility:>10.2%}")
print()
print(f"Total Trades:       {m.num_trades:>10}")
print(f"Win Rate:           {m.win_rate:>10.2%}")
print(f"Profit Factor:      {m.profit_factor:>10.2f}")
print(f"Avg Trade P&L:      ${m.avg_trade_pnl:>9.2f}")
print("=" * 70)

PERFORMANCE SUMMARY
Total Return:          -22.12%
Annualized Return:     -53.45%
Sharpe Ratio:           -1.159
Sortino Ratio:          -1.869
Max Drawdown:           36.04%
Volatility:             46.12%

Total Trades:               54
Win Rate:               39.29%
Profit Factor:            1.07
Avg Trade P&L:      $     2.03


## Next Steps

- **02_data_exploration.ipynb**: Explore data and quality checks
- **03_factor_research.ipynb**: Develop and analyze custom factors
- **04_strategy_comparison.ipynb**: Compare multiple strategies
- **05_parameter_optimization.ipynb**: Find optimal parameters